In [50]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import sklearn as skl
import pandas as pd
import tensorflow as tf

In [51]:
# Import Hg dataset
Hg_D = pd.read_csv('../Data/Hg_D.csv')
Hg_D.head()

,Unnamed: 0,MMHg_pM,PRES,TEMP,PSAL,DOXY,NITRAT
0,0,0.012,20.2,-0.7891,29.4120,433.4,0.33
1,1,0.012,20.2,-0.7891,29.4120,433.4,0.01
2,2,0.026,44.3,-1.2086,31.7411,399.2,6.80
3,3,0.026,44.3,-1.2086,31.7411,398.4,5.10
4,4,0.045,80.7,-1.5536,32.3635,318.8,11.79


In [52]:
# Drop Unnamed column
Hg_D.drop(columns=['Unnamed: 0'], inplace=True)
Hg_D.head()

,MMHg_pM,PRES,TEMP,PSAL,DOXY,NITRAT
0,0.012,20.2,-0.7891,29.4120,433.4,0.33
1,0.012,20.2,-0.7891,29.4120,433.4,0.01
2,0.026,44.3,-1.2086,31.7411,399.2,6.80
3,0.026,44.3,-1.2086,31.7411,398.4,5.10
4,0.045,80.7,-1.5536,32.3635,318.8,11.79


In [66]:
# Split our preprocessed data into our features and target arrays
X = Hg_D.drop(['MMHg_pM'], axis=1)
y = Hg_D['MMHg_pM']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

In [67]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [74]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=5,
        step=1), activation=activation, input_dim=5))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="relu"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [75]:
# Import the kerastuner library
import kerastuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


In [76]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 61 Complete [00h 00m 02s]
val_accuracy: 0.09215017408132553

Best val_accuracy So Far: 0.09215017408132553
Total elapsed time: 00h 02m 09s
INFO:tensorflow:Oracle triggered exit


In [77]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 9,
 'num_layers': 4,
 'units_0': 7,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0,
 'units_1': 1,
 'units_2': 1,
 'units_3': 1}

In [78]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 0.7802 - accuracy: 0.0922
Loss: 0.7802497148513794, Accuracy: 0.09215017408132553
